<a href="https://colab.research.google.com/github/plaban1981/BERT-NLP/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [0]:
import numpy as np
import pandas as pd
import re
import math
from bs4 import BeautifulSoup
import random
from google.colab import drive

In [2]:
!pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.2.0rc2)


In [2]:
import tensorflow as tf
tf.__version__

'2.2.0-rc2'

In [3]:
!pip install bert-for-tf2

     |████████████████████████████████| 40kB 4.3MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.1-cp36-none-any.whl size=30083 sha256=ffa94697cc566fea53be2c0febb20da18e226e3019ff552c91b7be665eb59ac9
  Stored in directory: /root/.cache/pip/wheels/dd/f1/10/861fd7899727e4034293fb1dfef45b00f8cd476d21d3b3821e
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=f76b7932ee846d461f517906f1a5a64d44dadc8795494d29281ab445d24ba421
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.0-cp36-none-any.whl size=15999 sha256=e32ea9e858cf699998b25f4867fe4973edf89ee807e3b6041eb4f5eace272e27
  Stored in directory: /root/.cache/pip/wheels/88/41/05/1a9955d1d01575bbd58aab76e22f8c7eeabba905d551576f43
Successfully built bert-for-tf2 py-params params-flow


In [5]:
!pip install sentencepiece

     |████████████████████████████████| 1.0MB 12.9MB/s 


In [6]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

2.2.0-rc2


## Loading Files

In [7]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
!unzip '/content/drive/My Drive/sentiments/trainingandtestdata.zip'


Archive:  /content/drive/My Drive/sentiments/trainingandtestdata.zip
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv  


In [0]:
clos = ['sentiment','id','date','query','user','text']

In [10]:
data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',
                   header=None,names=clos,engine='python',encoding='latin1')
data.head()

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [0]:
data.drop(['id','date','query','user'],axis=1,inplace=True)

In [12]:
data.shape

(1600000, 2)

In [13]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


## Data Preprocessing

In [0]:
def clean_tweet(tweet):
  tweet = BeautifulSoup(tweet,'lxml').get_text()
  tweet = re.sub(r"@[A-Za-z0-9]+",' ',tweet)
  tweet = re.sub(r"http?s://[A-Za-z0-9./]+",' ',tweet)
  tweet = re.sub(r"[^a-zA-Z.!?']",' ',tweet)
  tweet = re.sub(r" +",' ',tweet)
  return tweet

In [0]:
data['cleaned_text'] = data['text'].apply(lambda x : clean_tweet(x))

In [16]:
data.head()

,sentiment,text,cleaned_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",http twitpic.com y zl Awww that's a bummer. Y...
1,0,is upset that he can't update his Facebook by ...,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...,I dived many times for the ball. Managed to s...
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all....",no it's not behaving at all. i'm mad. why am ...


In [17]:
data.sentiment.value_counts()

4    800000
0    800000
Name: sentiment, dtype: int64

## converting positive sentiments from 4 to 1

In [0]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

## Tokenization

In [0]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
# all pretrained models are stored in the hub,12 stack of encoders
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
bert_layer = hub.KerasLayer(module_url,trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file,do_lower_case)

In [20]:
tokenizer.tokenize('I am going to the auditorium')

['i', 'am', 'going', 'to', 'the', 'auditorium']

In [21]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize('I am going to the auditorium'))

[1045, 2572, 2183, 2000, 1996, 11448]

In [0]:
def encode_sentence(sent):
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

In [0]:
data_cleaned = data.cleaned_text.values
data_inputs = [encode_sentence(sent) for sent in data_cleaned]

In [24]:
data_inputs[:1]

[[8299,
  1056,
  9148,
  25856,
  2594,
  1012,
  4012,
  1061,
  1062,
  2140,
  22091,
  2860,
  2860,
  2008,
  1005,
  1055,
  1037,
  26352,
  5017,
  1012,
  2017,
  2323,
  2050,
  2288,
  2585,
  12385,
  1997,
  2353,
  2154,
  2000,
  2079,
  2009,
  1012,
  1040]]

## Dataset Creation

create padded batches . Add padding at the end of all the sentences so that all the sentences are of the same length

In [0]:
data_with_len = [[sent,data_labels[i],len(sent)] for i,sent in enumerate(data_inputs)]
#shuffle the data so that we will have a combination of positive and negative tweets
random.shuffle(data_with_len)
#sort data_with_len on the basis of data length
data_with_len.sort(key=lambda x:x[2])
# not considering sentences with length < 7
sorted_all = [(sent_lab[0],sent_lab[1]) for sent_lab in data_with_len if sent_lab[2] > 7]

In [26]:
sorted_all[:2]

[([5962, 2000, 14085, 2332, 5624, 4823, 1999, 3009], 1),
 ([2987, 1005, 1056, 2031, 2115, 2171, 2006, 2009], 1)]

In [0]:
all_dataset = tf.data.Dataset.from_generator(lambda : sorted_all,output_types=(tf.int32,tf.int32))

In [28]:
next(iter(all_dataset))

(<tf.Tensor: shape=(8,), dtype=int32, numpy=
 array([ 5962,  2000, 14085,  2332,  5624,  4823,  1999,  3009],
       dtype=int32)>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [0]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,padded_shapes=((None,),()))

In [30]:
next(iter(all_batched))

(<tf.Tensor: shape=(32, 8), dtype=int32, numpy=
 array([[ 5962,  2000, 14085,  2332,  5624,  4823,  1999,  3009],
        [ 2987,  1005,  1056,  2031,  2115,  2171,  2006,  2009],
        [ 2300, 10199,  2239,  2003,  1996, 19690,  1997,  5909],
        [ 1045,  2031,  4743, 13433,  7361,  2006,  2026,  2067],
        [ 3403,  2005,  2026,  4536,  1012,  2821, 26095,  7501],
        [ 2074,  3427, 22162,  3892,  1012,  2204,  3185,   999],
        [ 1037, 11471,  2041,  5353,  6737,  2001,  3835,  1029],
        [ 1038, 13512,  2025,  1996,  2190,  2299,  2005,  2014],
        [ 2055,  2102,  2054, 25019,  1029,  6289,  2050,  1012],
        [ 1045,  2293,  2025,  2383,  2000,  2175,  2000,  2082],
        [14104,  6750,  5749,   999,  1045,  3335,  2383,  1043],
        [ 2085,  2042, 10443,  2115,  2061,  1056, 28394,  3215],
        [ 1045,  2123,  1005,  1056,  2113,  2054,  2000,  2360],
        [15854,  3287, 12687,  2836,  1012,  1012,  1012,  1012],
        [ 6293,  2216,  8265

In [0]:
NB_BATCHES = math.ceil(len(sorted_all)/BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

## Model Building

In [0]:
class DCNN(tf.keras.Model):
  def __init__(self,vocab_size,emb_dim=128,nb_filter=50,FFN_units=512,nb_classes=2,dropout_rate=0.1,training = False,name='dcnn'): 
               super(DCNN,self).__init__(name=name)
               self.embedding = layers.Embedding(vocab_size,emb_dim)
               self.bigram = layers.Conv1D(nb_filter,kernel_size=2,padding="valid",activation='relu')
               self.trigram = layers.Conv1D(nb_filter,kernel_size=3,padding="valid",activation='relu')
               self.fourgram = layers.Conv1D(nb_filter,kernel_size=4,padding="valid",activation='relu')
               self.pool = layers.GlobalMaxPooling1D()
               self.dense_1 = layers.Dense(units=FFN_units,activation='relu')
               self.dropout = layers.Dropout(rate=dropout_rate)
               if nb_classes == 2:
                 self.last_dense = layers.Dense(units=1,activation='sigmoid')
               else:
                 self.last_dense = layers.Dense(units=nb_classes,activation='softmax')


  def call(self,inputs,training):
    x = self.embedding(inputs)
    x_1 = self.bigram(x)
    x_1 = self.pool(x_1)
    x_2 = self.trigram(x)
    x_2 = self.pool(x_2)
    x_3 = self.fourgram(x)
    x_3 = self.pool(x_3) # shape = batch_size,filters
    merged = tf.concat((x_1,x_2,x_3),axis=1)
    merged = self.dense_1(merged)
    merged = self.dropout(merged,training)
    output = self.last_dense(merged)
    return output


  

## Training the Model

In [0]:
vocab_size = len(tokenizer.vocab)
emb_dim = 200
nb_filters = 100
FFN_units = 256
nb_classes = 2
dropout_rate = 0.2
nb_epochs = 5



In [0]:
dcnn = DCNN(vocab_size,emb_dim,nb_filters,FFN_units,nb_classes,dropout_rate)

In [48]:
dcnn

In [0]:
dcnn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [0]:
if nb_classes == 2:
  dcnn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
else :
   dcnn.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['sparse_categorical_accuracy'])

In [0]:
checkpoint_path = '/content/drive/My Drive/sentiments'
chkpoint = tf.train.Checkpoint(dcnn=dcnn)
ckpt_manager = tf.train.CheckpointManager(chkpoint,checkpoint_path,max_to_keep=1)

In [0]:
if ckpt_manager.latest_checkpoint:
  chkpoint.restore(ckpt_manager.latest_checkpoint)
  print('Latest Checkpoint restored')

In [0]:
## Custom Callback
class MycustomCallBack(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,los=None):
    ckpt_manager.save()
    print(f'Checkpoint saved at {checkpoint_path}')

In [65]:
dcnn.fit(train_dataset,epochs=5)

Epoch 1/5
37586/37586 [==============================] - 2122s 56ms/step - loss: 0.4284 - accuracy: 0.8031
Epoch 2/5
22940/37586 [=================>............] - ETA: 14:06 - loss: 0.3607 - accuracy: 0.8426Buffered data was truncated after reaching the output size limit.

## Evaluation

In [66]:
results = dcnn.evaluate(test_dataset)
print(results)

4176/4176 [==============================] - 23s 6ms/step - loss: 0.4286 - accuracy: 0.8331
[0.4285757839679718, 0.8330714106559753]


In [0]:
def get_predictions(sentence):
  tokens = encode_sentence(sentence)
  inputs = tf.expand_dims(tokens,0)
  output = dcnn(inputs,training=False)
  
  sentiment = np.round(output[0][0])
  print(sentiment)
  if sentiment == 0:
    print(f'output of the model {output} predicted sentiment : Negative')
  else:
    print(f'output of the model {output} predicted sentiment : Positive')

In [89]:
get_predictions('I hate you')

0.0
output of the model [[0.42882746]] predicted sentiment : Negative


In [90]:
get_predictions('I am very upset with the deeds of my friend')

0.0
output of the model [[1.3876527e-05]] predicted sentiment : Negative


In [91]:
get_predictions('I am feeling good')

1.0
output of the model [[0.9966845]] predicted sentiment : Positive
